In [1]:
import sys
sys.path.append('../../')

from coin_data import get_coin_data
from agents.rl_agent.util import eval_model, get_env, preprocess_data, train_model

In [2]:
%pip install gym gym-anytrading gym-trading-env stable_baselines stable-baselines3 sb3-contrib
from stable_baselines3 import DQN
from pandas._libs.tslibs.timestamps import Timestamp
from stable_baselines3.common.torch_layers import FlattenExtractor
from stable_baselines3.common.env_util import make_vec_env
from sb3_contrib import RecurrentPPO


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
data = get_coin_data('BTC/USDT', '1h', start_date=Timestamp('2021-01-01'))

In [4]:
data.head()

,Open,High,Low,Close,Volume
Timestamp,,,,,
2021-01-01 00:00:00,28923.63,29031.34,28690.17,28995.13,2311.811445
2021-01-01 01:00:00,28995.13,29470.00,28960.35,29409.99,5403.068471
2021-01-01 02:00:00,29410.00,29465.26,29120.03,29194.65,2384.231560
2021-01-01 03:00:00,29195.25,29367.00,29150.02,29278.40,1461.345077
2021-01-01 04:00:00,29278.41,29395.00,29029.40,29220.31,2038.046803


In [5]:
coin_data = preprocess_data(data, window_size=60)

In [6]:
coin_data.head()

,open,high,low,close,volume,feature_Close,feature_High,feature_Low,feature_Open,feature_Volume
Timestamp,,,,,,,,,,
2021-01-01 01:00:00,28995.13,29470.00,28960.35,29409.99,5403.068471,0.014308,1.002040,0.984711,0.985894,0.039379
2021-01-01 02:00:00,29410.00,29465.26,29120.03,29194.65,2384.231560,-0.007322,1.009269,0.997444,1.007376,0.017377
2021-01-01 03:00:00,29195.25,29367.00,29150.02,29278.40,1461.345077,0.002869,1.003026,0.995615,0.997160,0.010651
2021-01-01 04:00:00,29278.41,29395.00,29029.40,29220.31,2038.046803,-0.001984,1.005978,0.993467,1.001988,0.014854
2021-01-01 05:00:00,29220.31,29235.28,29084.11,29187.01,1469.956262,-0.001140,1.001654,0.996474,1.001141,0.010713


In [7]:
from environments.kucoin_trading_environment import KuCoinTradingEnvironment

kucoin_env = KuCoinTradingEnvironment(coin_data).spec

env = get_env(
    kucoin_env,
    'BTC/USDT', 
    coin_data, 
    window_size=60,
    positions=[-3, -2, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 2, 3],
    max_episode_duration=24 * 7,
)

TypeError: KuCoinTradingEnvironment.__init__() missing 1 required positional argument: 'df'

In [ ]:
env.reset()
for _ in range(100):
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    if done or truncated:
        print("info:", info)
        break
if not done or not truncated:
    print("info:", info)

info: {'idx': 19250, 'step': 100, 'date': numpy.datetime64('2023-03-14T16:00:00.000000000'), 'position_index': 2, 'position': -1, 'real_position': -0.9978843466798704, 'data_low': 25715.63, 'data_close': 25867.21, 'data_high': 26001.81, 'data_volume': 27079.12535, 'data_open': 25894.67, 'portfolio_valuation': 812.6093703391895, 'portfolio_distribution_asset': 0, 'portfolio_distribution_fiat': 1623.4995409660523, 'portfolio_distribution_borrowed_asset': 0.03134809428475854, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 9.404428285427562e-08, 'portfolio_distribution_interest_fiat': 0.0, 'reward': -0.001935531991102202}


In [ ]:
env.unwrapped.save_for_render(dir = "render_logs")

In [ ]:
from gym_trading_env.renderer import Renderer
renderer = Renderer(render_logs_dir="render_logs/BTC/")
renderer.run()

 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Jan/2024 17:28:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2024 17:29:05] "GET /update_data/USDT_2023-12-30_15-44-07.pkl HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2024 17:29:05] "GET /metrics HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2024 17:29:05] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Jan/2024 17:29:11] "GET /update_data/USDT_2024-01-02_17-28-48.pkl HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2024 17:29:11] "GET /metrics HTTP/1.1" 200 -


In [ ]:
vec_env = make_vec_env(lambda: env, n_envs=1)

In [ ]:
policy_args = {
    'normalize_images': False,
    'lstm_hidden_size': 64,
}
model = RecurrentPPO(
    'MlpLstmPolicy', 
    vec_env, 
    verbose=1, 
    n_steps=24*7,
    policy_kwargs=policy_args
)

Using cuda device


In [ ]:
# TODO: different way to calculate reward, should not be instantenous reward. Use margin, with take profit, take loss instead. Reward should be calculated at the end of the episode.
train_model('RecurrentPPO', model, env, 1e5)

Market Return : -3.59%   |   Portfolio Return : -27.94%   |   Position Changes : 158   |   Episode Length : 168   |   
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 167      |
|    ep_rew_mean     | -0.328   |
| time/              |          |
|    fps             | 277      |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 168      |
---------------------------------
Market Return : 26.79%   |   Portfolio Return : -36.15%   |   Position Changes : 155   |   Episode Length : 168   |   
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 167          |
|    ep_rew_mean          | -0.388       |
| time/                   |              |
|    fps                  | 141          |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 336          |
| train/                  |        

In [ ]:
eval_model('RecurrentPPO', model, env, coin_data, render=True, load_best=True)

{'idx': 12160, 'step': 0, 'date': numpy.datetime64('2022-05-23T06:00:00.000000000'), 'position_index': 6, 'position': 0.0, 'real_position': 0.0, 'data_low': 30284.91, 'data_close': 30487.77, 'data_high': 30655.0, 'data_volume': 2977.24082, 'data_open': 30301.46, 'portfolio_valuation': 1000.0, 'portfolio_distribution_asset': 0, 'portfolio_distribution_fiat': 1000.0, 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0, 'portfolio_distribution_interest_fiat': 0, 'reward': 0}
Market Return : -0.08%   |   Portfolio Return : -0.09%   |   Position Changes : 1   |   Episode Length : 168   |   
{'idx': 12327, 'step': 167, 'date': numpy.datetime64('2022-05-30T05:00:00.000000000'), 'position_index': array(8), 'position': 0.5, 'real_position': 0.4997893350370289, 'data_low': 30297.05, 'data_close': 30462.09, 'data_high': 30479.87, 'data_volume': 2574.84692, 'data_open': 30332.83, 'portfolio_valuation': 999.0788080747207,

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Jan/2024 22:42:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2024 22:42:35] "GET /update_data/USDT_2023-12-30_16-30-52.pkl HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2024 22:42:35] "GET /metrics HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2024 22:42:47] "GET /update_data/USDT_2024-01-02_22-42-28.pkl HTTP/1.1" 200 -
127.0.0.1 - - [02/Jan/2024 22:42:47] "GET /metrics HTTP/1.1" 200 -


!!! Server running http://127.0.0.1:5000 !!!


## Implement gym-trading env
https://gym-trading-env.readthedocs.io/en/latest/rl_tutorial.html

In [ ]:
## Multi Dataset Trading environment
# TODO: add data from more exchanges

## Implement own DQN model

In [ ]:
data = get_coin_data('BTC/USDT', '1h', start_date=Timestamp('2021-01-01'))

coin_data = preprocess_data(data, window_size=60)

env = get_env(
    'BTC/USDT', 
    coin_data, 
    window_size=60, 
    positions=[-3, -2, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 2, 3],
)

In [ ]:
# TODO: looks like the architecture is made specifically for training on images, and theres not much freedom to change the NN
# TODO: build own DQN model
# MlpPolicy aka DQNPolicy(
#   observation_space, 
#   action_space, 
#   lr_schedule, 
#   net_arch=None, 
#   activation_fn=<class 'torch.nn.modules.activation.ReLU'>, 
#   features_extractor_class=<class 'stable_baselines3.common.torch_layers.FlattenExtractor'>, 
#   features_extractor_kwargs=None, 
#   normalize_images=True, 
#   optimizer_class=<class 'torch.optim.adam.Adam'>, 
#   optimizer_kwargs=None)
dqn_args = {
    'normalize_images': False,
    'features_extractor_class': FlattenExtractor,
}
model = DQN('MlpPolicy', env, policy_kwargs=dqn_args, learning_starts=100, verbose=1, tau=0.9, exploration_initial_eps=0.5, exploration_fraction=0.1, exploration_final_eps=0.05)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
train_model('DQN', model, env, 1e5)

Market Return : 23.03%   |   Portfolio Return : -100.00%   |   Position Changes : 23265   |   Episode Length : 26156   |   
Eval num_timesteps=1000, episode_reward=-40.13 +/- 0.00
Episode length: 26155.00 +/- 0.00
----------------------------------
| eval/               |          |
|    mean_ep_length   | 2.62e+04 |
|    mean_reward      | -40.1    |
| rollout/            |          |
|    exploration_rate | 0.455    |
| time/               |          |
|    total_timesteps  | 1000     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00163  |
|    n_updates        | 449      |
----------------------------------
New best mean reward!
Market Return : 23.03%   |   Portfolio Return : -100.00%   |   Position Changes : 23123   |   Episode Length : 26156   |   
Eval num_timesteps=2000, episode_reward=-48.09 +/- 0.00
Episode length: 26155.00 +/- 0.00
----------------------------------
| eval/               |          |
|    mean_ep_length   | 2

In [ ]:
eval_model('DQN', model, env, coin_data, render=True, load_best=True)

{'idx': 59, 'step': 0, 'date': numpy.datetime64('2021-01-03T12:00:00.000000000'), 'position_index': 4, 'position': -0.5, 'real_position': -0.5, 'data_volume': 4546.283481, 'data_high': 34600.0, 'data_close': 34103.72, 'data_open': 34413.53, 'data_low': 33928.75, 'portfolio_valuation': 10000.0, 'portfolio_distribution_asset': 0, 'portfolio_distribution_fiat': 15000.0, 'portfolio_distribution_borrowed_asset': 0.14661157199273275, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0, 'portfolio_distribution_interest_fiat': 0, 'reward': 0}


Market Return : 23.03%   |   Portfolio Return : -100.00%   |   Position Changes : 10385   |   Episode Length : 26156   |   
{'idx': 26214, 'step': 26155, 'date': numpy.datetime64('2023-12-29T21:00:00.000000000'), 'position_index': array(11), 'position': 2, 'real_position': 2.0006947610235417, 'data_volume': 1535.9858, 'data_high': 42240.0, 'data_close': 41956.67, 'data_open': 42026.97, 'data_low': 41871.24, 'portfolio_valuation': 0.0008891038400504145, 'portfolio_distribution_asset': 4.23967248781845e-08, 'portfolio_distribution_fiat': 0, 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_borrowed_fiat': 0.0008897188855877059, 'portfolio_distribution_interest_asset': 0.0, 'portfolio_distribution_interest_fiat': 2.6691566567631173e-09, 'reward': -0.0033471093622339776}
 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Dec/2023 12:22:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 12:22:53] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Dec/2023 12:22:54] "GET /update_data/USDT_2023-12-13_01-32-31.pkl HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 12:22:54] "GET /metrics HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 12:23:22] "GET /update_data/USDT_2023-12-13_01-28-05.pkl HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 12:23:23] "GET /metrics HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 12:23:24] "GET /update_data/USDT_2023-12-29_23-47-42.pkl HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2023 12:23:24] "GET /metrics HTTP/1.1" 200 -


!!! Server running http://127.0.0.1:5000 !!!


## Implement PPO agent

In [ ]:
from stable_baselines3 import PPO

env = get_env(
    'BTC/USDT', 
    coin_data, 
    window_size=60, 
    positions=[-3, -2, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 2, 3],
)
vec_env = make_vec_env(lambda: env, n_envs=4)

model = PPO("MlpPolicy", vec_env, verbose=1)

# model = DQN('MlpPolicy', policy_kwargs= env, learning_starts=100, verbose=1, tau=0.9, exploration_initial_eps=0.5, exploration_fraction=0.1, exploration_final_eps=0.05)


TODO:
- observe copilot's demoralizing tips when it comes to trading 🤔🤔🤔 
- implement agent network. No idea what the openAI network looks like
- consider Random Forest model
- consider XGBoost model
- time seried model (ARIMA, GARCH)
- stacking multiple diverse models, and a meta model on top of that
- feed the results of a time series model as input features for a machine learning model
- Markov chain model
- consider RNN (but computationally expensive)
- consider transformer (works better with indicators)
- simple PPO
- consider PPO + LSTM model, https://sb3-contrib.readthedocs.io/en/master/modules/ppo_recurrent.html, https://iclr-blog-track.github.io/2022/03/25/ppo-implementation-details/

## Implement Random forest model

In [ ]:
# create labels for the data
#   want agent to take action in [-3, -2, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 2, 3] range, based on the price change
#   if price change is between -0.25% and 0.25%, then the label is 0
#   

In [ ]:
%pip install scikit-learn 
from sklearn.ensemble import RandomForestClassifier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 3.8 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 4.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 3.9 MB/s eta 0:00:000:00:01m eta 0:00:01

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# get train, test data
# get agent
random_forest_agent = 

# train agent
# test agent
# evaluate agent



env = get_env(
    'BTC/USDT',
    coin_data,
    window_size=60,
    positions=[-3, -2, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 2, 3],
)

random_forest_agent = 